In [ ]:
from fastscape.models import sediment_model
import xsimlab as xs
import xarray as xr
import numpy as np

import zarr
from ipyfastscape import TopoViz3d
import matplotlib.pyplot as plt
#plt.style.use('dark_background')
import hvplot.xarray

import dask

In [ ]:
sediment_model.visualize(show_inputs=True)

**Source to Sink model**

We predict the evolution of a source to sink system using the FastScape library. For this, we model an area experiencing uplift in one third of the modeled area and subsidence in the other two thirds. This represent a system composed of an uplifting mountain and an adjacent subsiding foreland basin. The uplift is uniform in the mountain while the subsidence in the basin decreases linearly from the mountain front to the edge of the basin/model.

The flux computed from FastScape will then be used as input for another (more recent) 1D marine deposition model that allows to reproduce more accurately deep water sedimentation. Note that this model is in the process of being published by its author, Charlie Shobe. You should therefore refrained from copying/distributing it.

With this setup we will investigate how a cyclic variation in precipitation rate is transmitted form the mountain area to the sedimentary basin and beyond.


**Periodic_Forcing Process**

We create a process to allow for the parameters depending on precipitation (Kf and G) to be made periodic functions of time.<br>
The inputs are a period and amplitude.

In [ ]:
from fastscape.processes import (DifferentialStreamPowerChannelTD)

@xs.process
class Periodic_Forcing:

    period = xs.variable(intent="inout", description="period of relative precipitation rate", attrs={"units": "yrs"})
    amplitude = xs.variable(intent="inout", description="amplitude relative precipitation rate", attrs={"units": "dimensionless"})
    
    k0_coef_bedrock = xs.variable(intent="in", description="erodibility (rate coefficient) for bedrock", attrs={"units": "m^(2-m)/yr"})
    k0_coef_soil = xs.variable(intent="in", description="erodibility (rate coefficient) for soil", attrs={"units": "m^(2-m)/yr"})
    g0_coef_bedrock = xs.variable(intent="in", description="transport coefficient for bedrock", attrs={"units": "dimensionless"})
    g0_coef_soil = xs.variable(intent="in", description="transport coefficient for bedrock", attrs={"units": "dimensionless"})

    m = xs.foreign(DifferentialStreamPowerChannelTD, 'area_exp', intent='in')
    k_coef_bedrock = xs.foreign(DifferentialStreamPowerChannelTD, 'k_coef_bedrock', intent='out')
    k_coef_soil = xs.foreign(DifferentialStreamPowerChannelTD, 'k_coef_soil', intent='out')
    g_coef_bedrock = xs.foreign(DifferentialStreamPowerChannelTD, 'g_coef_bedrock', intent='out')
    g_coef_soil = xs.foreign(DifferentialStreamPowerChannelTD, 'g_coef_soil', intent='out')

    rate = xs.variable(dims=(), static=False, intent="out", description="relative precipitation rate", attrs={"units": "dimensionless"})

    @xs.runtime(args="nsteps")
    def initialize(self, nsteps):
        self.rate = np.empty(nsteps)
    
    @xs.runtime(args=("step_start","step"))
    def run_step(self, tim, iout):
        precip = (1+self.amplitude + self.amplitude*np.sin(2*np.pi*tim/self.period))**5
        self.k_coef_bedrock = self.k0_coef_bedrock*precip**self.m
        self.k_coef_soil = self.k0_coef_soil*precip**self.m
        self.g_coef_bedrock = self.g0_coef_bedrock/precip
        self.g_coef_soil = self.g0_coef_soil/precip
        self.rate = precip


**DualUplift Process**

We create a process to prescribe an uplift function made of an uplift block next to a subsiding basin.<br>
The inputs are an uplift rate, a subsidence rate and the position in the x-direction of the boundary between the two.

In [ ]:
from fastscape.processes import (BlockUplift, RasterGrid2D)

@xs.process
class DualUplift:

    up = xs.variable(intent="inout", description="uplift rate", attrs={"units": "m/yrs"})
    down = xs.variable(intent="inout", description="subsidence rate", attrs={"units": "m/yrs"})
    xlim = xs.variable(intent="inout", description="boundary between mountain and basin", attrs={"units": "m"})
        
    uplift_rate = xs.foreign(BlockUplift, 'rate', intent='out')
    x = xs.foreign(RasterGrid2D, 'x')
    y = xs.foreign(RasterGrid2D, 'y')

    def initialize(self):
        X, Y = np.meshgrid(self.x, self.y)
        self.uplift_rate= np.where(X > self.xlim, self.up, -X/self.xlim*self.down)

**Wells Process**

We create a process that's going to record topographic and uplift/subsidence rate information at all time steps (not just the output time steps) but only for a limited number of wells

In [ ]:
from fastscape.processes import (SurfaceTopography, RasterGrid2D)

@xs.process
class Wells:

    strati = xs.variable(dims=("wells",), static=False, intent="out", description="erosion rate", attrs={"units": "m/yr"})
    rate = xs.variable(dims=("wells",), static=False, intent="out", description="erosion rate", attrs={"units": "m/yr"})
    
    wellx = xs.variable(dims="well", intent="in")
    welly = xs.variable(dims="well", intent="in")
    
    x = xs.foreign(RasterGrid2D, 'x')
    y = xs.foreign(RasterGrid2D, 'y')
    h = xs.foreign(SurfaceTopography, 'elevation', intent="in")
    uplift_rate = xs.foreign(BlockUplift, 'rate', intent='in')

    @xs.runtime(args="nsteps")
    def initialize(self, nsteps):
        self.ix = np.searchsorted(self.x, self.wellx)
        self.iy = np.searchsorted(self.y, self.welly)
        self.nwells = len(self.ix)
        self.rate = np.zeros((self.nwells,nsteps))
        self.strati = np.zeros((self.nwells,nsteps))

    def run_step(self):
        self.strati = self.h[self.iy, self.ix]
        self.rate = self.uplift_rate[self.iy, self.ix]
        

# FLux Process

To output the flux out of the model for input into the new 1D marine model developed by Charlie Sobe

In [ ]:
from fastscape.processes import (SurfaceTopography, RasterGrid2D)

@xs.process
class Flux:
    
    out = xs.variable(dims=(), static=False, intent="out", description="sediment flux", attrs={"units": "km^3/yr"})
    h = xs.foreign(SurfaceTopography, 'elevation', intent="in")
    u = xs.foreign(DualUplift, 'uplift_rate', intent="in")
    a = xs.foreign(RasterGrid2D, 'cell_area', intent="in")

    @xs.runtime(args="nsteps")
    def initialize(self, nsteps):
        self.out = np.zeros(nsteps)
        self.prev_dh = 0

    @xs.runtime(args=("step_delta"))
    def run_step(self, dt):
        dh = np.sum(self.h)
        self.out = (np.sum(self.u) - (dh-self.prev_dh)/dt)*self.a
#        print(self.a, (dh - self.prev_dh)/dt, np.sum(self.u))
        self.prev_dh = dh



**Model build**

We build the model by removing the diffusion component, adding our new process (SPL_Parameters) and selecting multiple flow direction for the drainage area computation (with $p=1$).

In [ ]:
from fastscape.processes import (MultipleFlowRouter)

TwoD_model = sediment_model.drop_processes('diffusion').update_processes({'forcing': Periodic_Forcing,
                                                                          'dualuplift': DualUplift, 'wells': Wells,
                                                                          'flow': MultipleFlowRouter,
                                                                          'flux': Flux})

TwoD_model.visualize()

**Model setup**

The model is size $nx\times nx$ and the first $nxb$ nodes are set to uplift while the others are subsiding.<br>
Number of time steps is $ntime$ and the number of output is $nout$. Total model run is 10 Myr.<br>
Numbr of wells is nwell

In [ ]:
nx = 101
ny = 51
nxb = int(2*nx/3)

xl = 100e3
yl = 50e3
xlim = xl*2/3

ntime = 1001
nout = 101
nwell = 9


**Model input**

Model input is built using input parameters and others, such as the reference values for Kf, G, m and n, and the size of the model (100x100 km).<br>
Boundary conditions are cyclic in the $y$-direction, no flux at $x=0$ and fixed (base level) at $x=L$.

Now that we have defined a function that takes the period of the climate variations as an input parameter, we can use xsimlab's batch computing optino. If one of the input variables is defined by an array of values with the dimension "batch", at execution, the model will be run as many times as the length of the batch dimension. This will also be done in parallel (using different processors/cores on the computer) if the *parallel* option is activated in the run command. Note that for this to happen, we must import/install *dask*.

In [ ]:
in_2D_ds = xs.create_setup(
    model=TwoD_model,
    clocks={
        'time': np.linspace(0, 1e7, ntime),
        'out': np.linspace(0, 1e7, nout)
    },
    master_clock='time',
    input_vars={
        'grid__shape': [ny, nx],
        'grid__length': [yl, xl],
        'boundary__status': ['fixed_value','core','looped','looped'],
        'dualuplift__up': 3e-3,
        'dualuplift__down': 1e-4,#('batch', 10**np.linspace(-5,-3,24)),
        'dualuplift__xlim': xlim,
        'forcing__period': ('batch', [2.e5,1e6,5.e6]),
        'forcing__amplitude': 0.5,
        'forcing__k0_coef_soil': 1e-5,
        'forcing__k0_coef_bedrock': 1e-5,
        'forcing__g0_coef_soil': 1,
        'forcing__g0_coef_bedrock': 1,
        'spl__slope_exp': 1,
        'spl__area_exp': 0.4,
        'flow__slope_exp': 1,
        'wells__wellx': np.linspace(0,xlim,nwell+2)[1:-1],
        'wells__welly': np.ones(nwell)*yl/2
    },
    output_vars={
        'topography__elevation':  'out',
        'drainage__area': 'out',
        'forcing__rate': 'time',
        'wells__rate': 'time',
        'wells__strati': 'time',
        'dualuplift__uplift_rate': 'out',
        'flux__out': 'time'
    }
)

In [ ]:
zgroup = zarr.group("out.zarr", overwrite=True)

**Model execution**

We run the model (which can take up to a few minutes depending on computer spped).

In [ ]:
#with xs.monitoring.ProgressBar():
out_2D_ds = in_2D_ds.xsimlab.run(model=TwoD_model, store=zgroup, batch_dim='batch', parallel=True, scheduler="processes")

**Display Results**

We display the model results in 3D including topography and drainage area.

In [ ]:
app = TopoViz3d(out_2D_ds, canvas_height=600, time_dim="out")

app.components['background_color'].set_color('lightgray')
app.components['vertical_exaggeration'].set_factor(5)
app.components['timestepper'].go_to_time(out_2D_ds.out[-1])

app.show()

**Compute stratigraphy**

We compute a stratigraphy along a cross-section at the center of the model.<br>
We also compute the flux out of the model (into the ocean), the flux out of the mountain and the maximum topography of the mountain as a function of time.
Note that the following graphic operations are performed for only one of the three batch models. To specify which one, use ibatch in the next cell.

In [ ]:
ibatch = 1
ymean = out_2D_ds.y.mean().values

In [ ]:
def compute_strati (ds, ymean):
    
    nout, nx = ds.topography__elevation.sel(y=ymean).values.shape
    strati = ds.topography__elevation.sel(y=ymean).values
    dt = (ds.out[1:].values - ds.out[:-1].values)

    for jout in range(1,nout):
        strati[:jout-1,:] = strati[:jout-1,:] + ds.dualuplift__uplift_rate.isel(out=jout).sel(y=ymean).values*dt[jout-1]

    for iout in range(nout-2, -1, -1):
        strati[iout,:] = np.minimum(strati[iout,:], strati[iout+1,:])

    return strati


In [ ]:
strati_2D = compute_strati (out_2D_ds.isel(batch=ibatch), ymean)

We plot the 2D stratigraphy in the middle of the basin, as a synthetic seismic line and as a series of wells, one every four nodes

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, sharex=False, sharey=True, figsize=(32,16))

nfreq_reflector = 1

colors = plt.cm.jet(np.linspace(0,1,nout))
for iout in range(nout-1, -1, -nfreq_reflector):
    ax[0].fill_between(out_2D_ds.sel(y=ymean).x[0:nxb], strati_2D[iout,0:nxb], strati_2D[0,0:nxb], color=colors[iout])
    ax[0].plot(out_2D_ds.sel(y=ymean).x[0:nxb], strati_2D[iout,0:nxb], color='darkgrey')

ymin = strati_2D[:,nxb-1].min()
ymax = strati_2D[:,nxb-1].max()
ax[0].set_ylim((ymin,ymax))

for iwell in range(0,nxb, 4):
    for iout in range(nout-1, -1, -nfreq_reflector):
        ax[1].fill_between((iwell, iwell+1), (strati_2D[iout,iwell],strati_2D[iout,iwell]), (strati_2D[0,iwell],strati_2D[0,iwell]), color=colors[iout])
        ax[1].plot((iwell, iwell+1), (strati_2D[iout,iwell],strati_2D[iout,iwell]), color='darkgrey')


We display the resulting stratigraphic column showing erosion rate estimated from layers thickness as a function of time. This is equivalent to assuming that we can measure the thickness of all preserved layers and assess their age with infinite precition

Function to compute sedimentation rate in a series of wells located in the basin

In [ ]:
def compute_well_strati (ds):
    
    ntime, nwell =  ds.wells__strati.shape
    strati = ds.wells__strati.values
    sedim_rate = np.empty((ntime-1, nwell))
    dt = (ds.time[1:].values - ds.time[:-1].values)

    for jout in range(1,ntime):
        strati[:jout-1,:] = strati[:jout-1,:] + ds.wells__rate.isel(time=jout).values*dt[jout-1]

    for iout in range(ntime-2, -1, -1):
        strati[iout,:] = np.minimum(strati[iout,:], strati[iout+1,:])

    for well in range(nwell):
        sedim_rate[:,well] = (strati[1:,well] - strati[:-1,well])/dt

    return sedim_rate


In [ ]:
sedim = compute_well_strati(out_2D_ds.isel(batch=ibatch))

Compute fluxes out of the system, out of the mountain, channel mobility and maximum topography as a function of time for display/comparison with well deposition rate

In [ ]:
u2D = out_2D_ds.dualuplift__uplift_rate.isel(batch=ibatch).values
topo = out_2D_ds.topography__elevation.isel(batch=ibatch).values
area = out_2D_ds.drainage__area.isel(batch=ibatch).values

flux_out_2D = np.sum(np.sum(u2D[1:,:,:],1),1)-np.sum(np.sum(topo[1:,:,:] - topo[:-1,:,:], 1), 1)/(out_2D_ds.out.values[1:] - out_2D_ds.out.values[:-1])
flux_out_mountain = np.sum(np.sum(u2D[1:,:,nxb:],1),1)-np.sum(np.sum(topo[1:,:,nxb:] - topo[:-1,:,nxb:], 1), 1)/(out_2D_ds.out.values[1:] - out_2D_ds.out.values[:-1])
channel_mobility = np.sum(np.abs(area[1:,:, int(nxb/2)]-area[:-1,:, int(nxb/2)]),1)/(out_2D_ds.out.values[1:] - out_2D_ds.out.values[:-1])

topo_2D = topo.max(1).max(1)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = nwell+4, sharex=False, sharey=True, figsize=(32,16))

precip = out_2D_ds.forcing__rate.isel(batch=ibatch)
tmax = out_2D_ds.time.values.max()
time = tmax - out_2D_ds.time.values
out = tmax - out_2D_ds.out.values

for well in range(nwell):
    kwell = well + 1
    ax[kwell].plot(precip/precip.max()*sedim[:,well].max(), time, alpha=0.5)
    ax[kwell].plot(sedim[:,well], time[:-1])
    ax[kwell].set_ylim(tmax,0)
    ax[kwell].set_title("Well "+str(kwell))

ax[0].plot(precip/precip.max()*flux_out_2D.max(), time, alpha=0.5)
ax[0].plot(flux_out_2D, out[:-1])
ax[0].set_ylim(tmax,0)
ax[0].set_title("Flux out")

ax[-3].plot(precip/precip.max()*channel_mobility.max(), time, alpha=0.5)
ax[-3].plot(channel_mobility, out[:-1])
ax[-3].set_ylim(tmax,0)
ax[-3].set_title("Channel mobility")

ax[-2].plot(precip/precip.max()*flux_out_mountain.max(), time, alpha=0.5)
ax[-2].plot(flux_out_mountain, out[:-1])
ax[-2].set_ylim(tmax,0)
ax[-2].set_title("Flux mountain")

ax[-1].plot(precip/precip.max()*topo_2D.max(), time, alpha=0.5)
ax[-1].plot(topo_2D, out)
ax[-1].set_ylim(tmax,0)
ax[-1].set_title("Topo")

ax[0].set_ylabel("Time b.p. (yr)");

In [ ]:
import Charlie_Model

In [ ]:
Charlie_Model.marine.visualize(show_inputs=True)

We now prepare a setup consisting of a basement geometry made of a steep ramp down next to a less steep ramp up.

We also transform the flux output into an output for the marine model.

We display both.

We also prepare a sea level array to transform the periodic climate fluctuations (forcing) into a sea level signal. We change the sign of the climate signal (high precipitation assumed during cold period and thus low sea level) but this could be reversed, depending on the region of Earth that is modeled.


In [ ]:
length = 1e6
spacing = length/nx

br = -np.ones((ntime, nx))*6e3
nxflat = int(nx/5)
br[:,:nxflat] = np.linspace(0,-6.e3,nxflat)
br[:,nxflat:] = np.linspace(-6.e3,-5.e3,nx-nxflat)

bedrock_elev_array = xr.DataArray(br, dims=['time', 'x'])
initial_bedrock = bedrock_elev_array[0, :]

qs_array = xr.DataArray(np.where(out_2D_ds.flux__out.isel(batch=ibatch).values>0,
                                 out_2D_ds.flux__out.isel(batch=ibatch).values,0), dims=['time'])

sea_level = xr.DataArray(-4*out_2D_ds.forcing__rate.isel(batch=ibatch).values, dims=['time'])

fig, ax = plt.subplots(nrows = 1, ncols = 2, sharex=False, sharey=False, figsize=(32,10))

ax[0].plot(initial_bedrock)
ax[1].plot(qs_array)

noutp = nout

Now we prepare the model setup and run it.

In [ ]:
in_ds = xs.create_setup(
   ...:     model=Charlie_Model.marine,
   ...:     clocks={
   ...:         'time': np.linspace(0, 1e7, ntime),
   ...:         'otime': np.linspace(0, 1e7, noutp)
   ...:     },
   ...:     master_clock='time',
   ...:     input_vars={
   ...:         'grid': {'length': length, 'spacing': spacing},
                'init': {'init_br': initial_bedrock},
                'erode': {
                    'k_factor': 0.2,
                    'k_depth_scale': 100.,
                    's_crit': 0.05,
                    'travel_dist': 200000.,
                    'sed_porosity': 0.56,
                    'sed_porosity_depth_scale': 2830.,
                    'sea_level': sea_level,
                    'qs_in': qs_array,
                    'basin_width': 1e5
                },
                'profile': {
                    'br': bedrock_elev_array
                },
   },
            output_vars={'profile__z': 'otime', 'profile__br': 'otime', 'profile__h': 'otime'}
)

In [ ]:
zgroup_marine = zarr.group("out_Marine.zarr", overwrite=True)


In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=Charlie_Model.marine, store=zgroup_marine)

We compute the 1D stratigraphy

In [ ]:
strati = np.zeros((noutp,nx))
for iout in range(noutp):
    strati[iout,:] = out_ds.profile__z.isel(otime=iout)
for iout in range(noutp-2, -1, -1):
    strati[iout,:] = np.minimum(strati[iout,:], strati[iout+1,:])


And plot it

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, sharex=False, sharey=False, figsize=(32,20))

colors = plt.cm.rainbow(np.linspace(0,1,noutp))

for iout in range(noutp-1, -1, -5):
    ax[0].fill_between(out_ds.x, out_ds.profile__br[iout,:], strati[iout,:], color=colors[iout])
    ax[0].plot(out_ds.x, strati[iout,:], color='darkgrey')
    
for iout in range(noutp-1, -1, -2):
    ax[1].fill_between(out_ds.x, out_ds.profile__br[iout,:], strati[iout,:], color=colors[iout])
    ax[1].plot(out_ds.x, strati[iout,:], color='darkgrey')
    
ax[1].set_xlim((0, 200e3))
ax[1].set_ylim((-1e3, 0));